In [2]:
import pandas as pd
import numpy as np
import os,re,csv,time
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import Imputer
from collections import Counter
pd.options.mode.chained_assignment = None
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV,Lasso,LassoLarsIC,ElasticNet,MultiTaskLasso,LassoLars
from sklearn.linear_model import BayesianRidge,SGDRegressor,Perceptron,PassiveAggressiveRegressor,RANSACRegressor,TheilSenRegressor
from sklearn.cross_validation import KFold,cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib
#****************************************Data Gathering
sfdata=pd.read_csv('C:/Users/TG/Downloads/State Farm Data Science WORK SAMPLE/TrainData.csv')
sftest=pd.read_csv('C:/Users/TG/Downloads/State Farm Data Science WORK SAMPLE/TestData.csv')
#****************************************Data Processing
#remove the $ & % values to convert text to number for variables X2,X4,X5,X6
sfdata.X4.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
sfdata.X5.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
sfdata.X6.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
sfdata.X1.replace(regex=True,inplace=True,to_replace=r'%',value=r'')
sfdata.X30.replace(regex=True,inplace=True,to_replace=r'%',value=r'')
#****************
sftest.X4.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
sftest.X5.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
sftest.X6.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
sftest.X1.replace(regex=True,inplace=True,to_replace=r'%',value=r'')
sftest.X30.replace(regex=True,inplace=True,to_replace=r'%',value=r'')

#removing the id columns X2,X3 as they dont add any value to prediction
sfdata.drop(sfdata.columns[[1,2]], axis=1, inplace=True)
sftest.drop(sftest.columns[[1,2]], axis=1, inplace=True)
#Removing the X16 as imputing will be take a complex process and will leave for some other time
sfdata.pop('X16')
sftest.pop('X16')
sfdata.pop('X10')
sftest.pop('X10')
#Converting the variables 'X1', 'X4','X5','X6','X29','X30' to float type
sfdata[['X1', 'X4','X5','X6','X29','X30']] = sfdata[['X1', 'X4','X5','X6','X29','X30']].astype(float)
sftest[['X1', 'X4','X5','X6','X29','X30']] = sftest[['X1', 'X4','X5','X6','X29','X30']].astype(float)

# Since X4,X5,X6 all are highly correlated with correlation score 0.99 score and leads to multi collinearity 
#I am removing taking out X4 & x6 and retaining X5
sfdata.pop('X4')
sfdata.pop('X6')
sftest.pop('X4')
sftest.pop('X6')

#convert categorical variables into numeric
# Pre-processing non-number values
le = LabelEncoder()
# 
for col in ['X7','X8','X9','X11','X12','X15','X17','X18','X19','X20','X14','X32','X23'] :
    mask = sfdata[col].notnull()
    mask1= sftest[col].notnull()
    le.fit(list(sfdata[col])+list(sftest[col]))
    print(col)    
    sfdata[col][mask] = le.transform(sfdata[col][mask])
    sftest[col][mask1] = le.transform(sftest[col][mask1])

#****************************************Data Imputing
#.Fit the knn model to predict X8 variable to impute X8 data    
features=['X5','X7','X11','X14','X15','X17','X19','X20','X22','X23','X24','X27','X28','X29','X31','X32']
X=sfdata[features][sfdata.X8.notnull()]
y=list(sfdata['X8'][sfdata.X8.notnull()])
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X,y)
sfdata['X8'][sfdata.X8.isnull()]=clf.predict(sfdata[features][sfdata.X8.isnull()])    

#.Fit the knn model to predict X9 variable to impute X9 data  
features=['X5','X8','X7','X11','X14','X15','X17','X19','X20','X22','X23','X24','X27','X28','X29','X31','X32']
X=sfdata[features][sfdata.X9.notnull()]
y=list(sfdata['X9'][sfdata.X9.notnull()])
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X,y)
sfdata['X9'][sfdata.X9.isnull()]=clf.predict(sfdata[features][sfdata.X9.isnull()])

#.Fit the knn model to predict X12 variable
features=['X5','X7','X11','X14','X15','X17','X19','X20','X22','X23','X24','X27','X28','X29','X31','X32']
X=sfdata[features][sfdata.X12.notnull()]
y=list(sfdata['X12'][sfdata.X12.notnull()])
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X,y)
sfdata['X12'][sfdata.X12.isnull()]=clf.predict(sfdata[features][sfdata.X12.isnull()])

#.Fit the knn model to predict X18 variable
features=['X5','X7','X11','X14','X15','X17','X19','X20','X22','X23','X24','X27','X28','X29','X31','X32']
X=sfdata[features][sfdata.X18.notnull()]
y=list(sfdata['X18'][sfdata.X18.notnull()])
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X,y)
sfdata['X18'][sfdata.X18.isnull()]=clf.predict(sfdata[features][sfdata.X18.isnull()])

#.fill the null values of 25 & 26 with 0
sfdata[['X25']]=sfdata[['X25']].fillna(0)
sfdata[['X26']]=sfdata[['X26']].fillna(0)
sftest[['X25']]=sftest[['X25']].fillna(0)
sftest[['X26']]=sftest[['X26']].fillna(0)
#.fill the null values of X30 to mean value, instead of 0 to avoid risk
sfdata[['X30']]=sfdata[['X30']].fillna(sfdata[['X30']].mean())
sftest[['X30']]=sftest[['X30']].fillna(sfdata[['X30']].mean())

X7
X8
X9
X11
X12
X15
X17
X18
X19
X20
X14
X32
X23


C:\Users\TG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
#************************************* Creating data set for training
#removing rows where X30,X13 as there null

train=sfdata[sfdata.X1.notnull()]
train=train[train.X13.notnull()]
#split the data into training and test sets
features=list(train.columns.values)
features.remove('X1')
#features.remove('X30')
X_Train,X_Test,Y_Train,Y_Test=train_test_split(train[features],train['X1'],test_size=0.33, random_state=42)

# Define classifiers
classifiers = [
    LinearRegression(),
    Ridge (alpha = .5),
    RidgeCV(alphas=[0.1, 1.0, 10.0]),
    Lasso(alpha = 0.1),
    LassoLarsIC(),
    ElasticNet(),    
    LassoLars(),
    BayesianRidge(),
    SGDRegressor(),     
]

# Train all the classifiers and evaluate results to pick the best
for classifier in classifiers:
    print (classifier.__class__.__name__)
    start = time.time()
    scores=-cross_val_score(classifier,X_Train,(Y_Train),cv=10,scoring='mean_squared_error')
    print('K-Fold cross validation to verify consistancy in the errors')
    print(np.sqrt(scores))
    classifier.fit(X_Train, Y_Train)
    joblib.dump(classifier, 'C:/Users/TG/Downloads/State Farm Data Science WORK SAMPLE/'+classifier.__class__.__name__+'.pkl')
    print ('  -> Training time:', time.time() - start)

# Evaluation and export result
for classifier in classifiers:
    print (classifier.__class__.__name__)
    #print ('Log Loss:')
    #print (log_loss(Y_Test, classifier.predict_proba(X_Test)))
    clf= joblib.load('C:/Users/TG/Downloads/State Farm Data Science WORK SAMPLE/'+classifier.__class__.__name__+'.pkl')
    print ('RMSE:')
    print (mean_squared_error(Y_Test, clf.predict(X_Test))**0.5) # RMSE
    print

    if not os.path.exists('result/'):
        os.makedirs('result/')
    predictions = np.column_stack((clf.predict(X_Test)))
    csvfile = 'result/' + classifier.__class__.__name__ + '-submit.csv'
    with open(csvfile, 'w') as output:
        writer = csv.writer(output, lineterminator='\n')
        writer.writerow(['id','intrestrate'])
        writer.writerows(predictions)

LinearRegression
K-Fold cross validation to verify consistancy in the errors
[ 2.0807242   2.06717801  2.07778884  2.05630847  2.05507581  2.02420517
  2.06717559  2.13268567  2.08565766  2.1101913 ]
  -> Training time: 10.138662576675415
Ridge
K-Fold cross validation to verify consistancy in the errors
[ 2.08072425  2.06717793  2.07778863  2.05630871  2.0550755   2.02420493
  2.0671757   2.132686    2.08565769  2.11019132]
  -> Training time: 6.448493480682373
RidgeCV
K-Fold cross validation to verify consistancy in the errors
[ 2.08133823  2.0679276   2.07785048  2.05643031  2.05520314  2.0435674
  2.06743381  2.13604821  2.08661063  2.11061678]
  -> Training time: 12.029312372207642
Lasso
K-Fold cross validation to verify consistancy in the errors
[ 2.11846821  2.09958041  2.10720546  2.09874012  2.0852247   2.05291712
  2.10831662  2.17864988  2.12459494  2.14636007]
  -> Training time: 11.657701969146729
LassoLarsIC
K-Fold cross validation to verify consistancy in the errors
[ 2.0

In [35]:
# Based on above results except for the SGDRegressor all other models are consistantly performing , 
#so I am choosing LinearRegression() & Bayesianridge() to fit entire test data and extract the results from test data
train=sfdata[sfdata.X1.notnull()]
train=train[train.X13.notnull()]
#split the data into training and test sets
features=list(train.columns.values)
features.remove('X1')

# Define classifiers
classifiers = [
    LinearRegression(),
    BayesianRidge()
]

# Train all the classifiers and evaluate results to pick the best
for classifier in classifiers:
    print (classifier.__class__.__name__)
    start = time.time()
    classifier.fit(train[features],train['X1'])
    joblib.dump(classifier, 'C:/Users/TG/Downloads/State Farm Data Science WORK SAMPLE/'+classifier.__class__.__name__+'_final.pkl')
    print ('  -> Training time:', time.time() - start)

    
#Load the model and predict the results and save them to a file    
for classifier in classifiers:
    print (classifier.__class__.__name__)
    clf= joblib.load('C:/Users/TG/Downloads/State Farm Data Science WORK SAMPLE/'+classifier.__class__.__name__+'_final.pkl')   

    if not os.path.exists('result/'):
        os.makedirs('result/')
    predictions = np.column_stack((clf.predict(sftest[features])))
    csvfile = 'result/' + classifier.__class__.__name__ + '-finalsubmit.csv'
    with open(csvfile, 'w') as output:
        writer = csv.writer(output, lineterminator='\n')
        writer.writerow(['intrest_rate'])
        writer.writerows(predictions)

LinearRegression
  -> Training time: 1.1909904479980469
BayesianRidge
  -> Training time: 1.8241329193115234
LinearRegression
BayesianRidge


In [5]:
sfdata['X1'].std()

4.3779445055781565